'''
Connor Smith
BMI559
HW1 - Late
'''

In [2]:
import csv
import pandas
import re

In [3]:
# import file
vcf_file = open("PTEN.vcf","r")

vcf_lines = vcf_file.readlines()
#strip lines
vcf_clean = [line.rstrip() for line in vcf_lines]

vcf_file.close()

Create Data Dictionary for INFO things

In [24]:
#create dictionary for data dict

data_dict = {}

# check for ##INFO lines
for line in vcf_clean:
    match = re.search(r"##INFO",line)
    if match:
        #print(line)
        #print(re.search(r"<(.*)>",line).group(1))
        attribs = re.search(r"<(.*)>",line).group(1).split(',')
        info_id = attribs[0][3:]
        info_def = attribs[3][12:].replace('"','')
        info_type = attribs[2][5:]
        # range to be added later
        info_range = ''
        data_dict[info_id] = [info_def,info_type,info_range]
print(data_dict)

{'CIEND': ['Confidence interval around END for imprecise variants', 'Integer', ''], 'CIPOS': ['Confidence interval around POS for imprecise variants', 'Integer', ''], 'CS': ['Source call set.', 'String', ''], 'END': ['End coordinate of this variant', 'Integer', ''], 'IMPRECISE': ['Imprecise structural variation', 'Flag', ''], 'MC': ['Merged calls.', 'String', ''], 'MEINFO': ['Mobile element info of the form NAME', 'String', ''], 'MEND': ['Mitochondrial end coordinate of inserted sequence', 'Integer', ''], 'MLEN': ['Estimated length of mitochondrial insert', 'Integer', ''], 'MSTART': ['Mitochondrial start coordinate of inserted sequence', 'Integer', ''], 'SVLEN': ['SV length. It is only calculated for structural variation MEIs. For other types of SVs; one may calculate the SV length by INFO:END-START+1', 'Integer', ''], 'SVTYPE': ['Type of structural variant', 'String', ''], 'TSD': ['Precise Target Site Duplication for bases', 'String', ''], 'AC': ['Total number of alternate alleles in 

Parse the data into a table

In [80]:
# ignore the specific mutation columns
data_raw = [line.split('\t') for line in vcf_clean[252:]]

In [118]:
list_of_dict = []
for row in data_raw[1:]:
    # extract info variables
    info = row[7]
    info_dict = {}
    for item in info.split(';'):
        match = re.search(r"=",item)
        if match:
            info_dict[item.split("=")[0]] = item.split("=")[1]
        else:
            # deals with FLAG columns
            info_dict[item] = 1
    #info_split = {item.split('=')[0]: item.split('=')[1] for item in info.split(";") if item.split('=')[0] != 'EX_TARGET'}
    row_dict = {'#CHROM':row[0],
               'POS':row[1],
               'ID':row[2],
               'REF':row[3],
               'ALT':row[4],
               'QUAL':row[5],
               'FILTER':row[6],
               'INFO':row[7],
               'FORMAT':row[8]}
    row_dict.update(info_dict)
    list_of_dict.append(row_dict)
PTEN_data = pandas.DataFrame(list_of_dict)

In [117]:
print('Number of SNPs: '+str(len(PTEN_data.loc[PTEN_data['VT'] == 'SNP'])))

Number of SNPs: 2710


In [145]:
# Set range/unique values for each column
for key, item in data_dict.items():
    if key in PTEN_data.columns:
        item[2] = str(len(PTEN_data[key].unique()))+' unique values'
    else:
        item[2] = 'No Values Found'

{'AA': ['Ancestral Allele. Format: AA|REF|ALT|IndelType. AA: Ancestral allele',
  'String',
  '50 unique values'],
 'AC': ['Total number of alternate alleles in called genotypes',
  'Integer',
  '197 unique values'],
 'AF': ['Estimated allele frequency in the range (0',
  'Float',
  '197 unique values'],
 'AFR_AF': ['Allele frequency in the AFR populations calculated from AC and AN',
  'Float',
  '129 unique values'],
 'AMR_AF': ['Allele frequency in the AMR populations calculated from AC and AN',
  'Float',
  '73 unique values'],
 'AN': ['Total number of alleles in called genotypes',
  'Integer',
  '1 unique values'],
 'CIEND': ['Confidence interval around END for imprecise variants',
  'Integer',
  '3 unique values'],
 'CIPOS': ['Confidence interval around POS for imprecise variants',
  'Integer',
  '3 unique values'],
 'CS': ['Source call set.', 'String', '4 unique values'],
 'DP': ['Total read depth; only low coverage data were counted towards the DP',
  'Integer',
  '2441 unique v

In [146]:
# Print Dictionary
dict_file = open('data_dictionary_PTEN.txt','w')
for key,item in data_dict.items():
    dict_file.write('TERM\tDEFINITION\tTYPE\tRANGE')
    dict_file.write(key+'\t'+item[0]+'\t'+item[1]+'\t'+item[2])
dict_file.close()